In [ ]:
import networkx as nx
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
import time


# Define the function to compute graph properties
def compute_properties(G):
    properties = {
        "Average degree": sum(dict(G.degree()).values()) / G.number_of_nodes(),
        "Max degree": max(dict(G.degree()).values()),
        "Density": nx.density(G),
        "Number of connected components": nx.number_connected_components(G) if not G.is_directed() else "N/A",
        "Efficiency": nx.global_efficiency(G),  # Alternative for average shortest path length for unconnected graphs
        "Transitivity": nx.transitivity(G),
        "Average Clustering Coefficient": nx.average_clustering(G),
        "Assortativity": nx.degree_assortativity_coefficient(G),
    }
    return properties


# Define a function for generating and computing properties of a random graph
def compute_random_graph_properties(args):
    G, seed = args
    p = nx.density(G)
    random_graph = nx.erdos_renyi_graph(n=G.number_of_nodes(), p=p, seed=seed)
    return compute_properties(random_graph)


# Main script
if __name__ == "__main__":
    # Example graph
    G = nx.karate_club_graph()

    # Measure time for the main graph
    start_time = time.time()
    properties_G = compute_properties(G)
    end_time = time.time()
    print(f"Time to compute properties for graph G: {end_time - start_time}")

    # Parallelized computation of random graph properties
    num_random_graphs = 3
    seeds = range(num_random_graphs)

    start_time = time.time()
    with Pool(cpu_count()-3) as pool:
        # Map inputs (G and seed) to the worker function
        random_properties_list = pool.map(compute_random_graph_properties, [(G, seed) for seed in seeds])

    end_time = time.time()
    print(f"Time to compute properties for random graphs: {end_time - start_time}")

    # Compute averages of random graph properties
    random_properties_avg = pd.DataFrame(random_properties_list).mean(numeric_only=True).to_dict()

    # Compare properties
    comparison = pd.DataFrame({"G Graph": properties_G, "Erdős-Rényi Avg": random_properties_avg})
    display(comparison)


Time to compute properties for graph G: 0.01959991455078125
